In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
import heapq
from keybert import KeyBERT
import nltk

In [21]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        list_word = [list_word]
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        return self.keyword_value
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [23]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  body_tag = soup.find('body')
  text_below_body = body_tag.get_text() 
  body_list =[]
  body_list.append(text_below_body)
  return (body_list,title_tag)

In [31]:
#check language function
def check_lang(url:str):
    data_lang = scrape_tags(url)
    percent = pythainlp.util.countthai(data_lang)
    print(data_lang)
    print(percent)
    

In [32]:
check_lang('https://www.bbc.com/afrique/media/photogalleries')

BBC News, AfriqueAller au contenuRubriquesPage en cours, AccueilAfriqueMondeSantéBien-êtreScienceTechnologieEconomieCultureVidéosNos émissionsEcoutez en directPage en cours, AccueilAfriqueMondeSantéBien-êtreScienceTechnologieEconomieCultureVidéosNos émissionsEcoutez en directBBC News, Afrique - AccueilÀ la uneCamilla Pang, autiste, biochimiste auteur d'un livre qui transcende l'humainÀ 28 ans, Camilla est devenue la plus jeune lauréate d'un prix décerné auparavant aux scientifiques tels que Stephen Hawking.il y a 2 heuresLire plusComment la biologie quantique peut fournir des indices sur la raison pour laquelle nous sommes vivants ?L'histoire étonnante de la scientifique qui, suite à un accident vasculaire cérébral qui l'a temporairement paralysée, a fait une grande découvertePourquoi le génome humain n'a jamais été entièrement déchiffréLe déchiffrement du génome humain est considéré par les experts comme l'un des jalons scientifiques les plus importants de l'histoire.il y a 5 heuresCo

In [33]:
check_lang('https://www.thairath.co.th/news/crime/2633816')


ไทยรัฐ สำนักข่าวอันดับ 1 ของไทย | ไทยรัฐออนไลน์ไทยรัฐออนไลน์ข่าวพระราชสำนักทั่วไทยในกระแสการเมืองเศรษฐกิจต่างประเทศอาชญากรรมยานยนต์เทคโนโลยีราคาทองคำรายงานพิเศษวิดีโอหนังสือพิมพ์ไทยรัฐทีวีดูย้อนหลังผังรายการLiveไลฟ์สไตล์กีฬาฟุตบอลต่างประเทศฟุตบอลไทยSport insiderไฟต์สปอร์ตกีฬาโลกวิดีโอแกลเลอรี่ฟุตบอลโลก 2022บันเทิงดวงหวยนิยายโปรโมชั่นซื้อ-ขายส่วนลดเช็คราคาThairathPlusยิงดับ "พล.ต.ท.ปัญญา" ผบช.น.ยันฝีมือภรรยา อยู่ในอาการช็อก ยังให้การไม่ได้LiveLiveเข้าสู่ระบบสมัครสมาชิกค้นหาHamburgerHamburgerค้นหาTHAIRATH MEMBERSHIPเข้าสู่ระบบสมัครสมาชิกLive ชมรายการสดข่าวBackข่าวข่าวล่าสุดพระราชสำนักทั่วไทยในกระแสการเมืองเศรษฐกิจต่างประเทศอาชญากรรมยานยนต์เทคโนโลยีราคาทองคำรายงานพิเศษกีฬาBackฟุตบอลต่างประเทศฟุตบอลไทยSport insiderไฟต์สปอร์ตกีฬาโลกวิดีโอแกลเลอรี่ฟุตบอลโลก 2022ไทยรัฐทีวีBackดูย้อนหลังวาไรตี้บันเทิงกีฬาผังรายการLiveโปรโมชั่นBackโปรโมชั่นซื้อ-ขายส่วนลดเช็คราคาThairath Talkหนังสือพิมพ์คอลัมน์บันเทิงดวงหวยนิยายวิดีโอPodcastไลฟ์สไตล์กิจกรรมThairath PlusBackThairath PlusSpeakMoneyEveryday LifeNat